In [1]:
# --- 라이브러리 임포트 ---
import numpy as np
import random
import torch

# --- 시드 고정 ---
# PyTorch의 CPU 연산 시드 고정
torch.manual_seed(42)
# PyTorch의 GPU 연산 시드 고정
torch.cuda.manual_seed(42)
# NumPy의 난수 시드 고정
np.random.seed(42)
# Python 내장 random 모듈의 시드 고정
random.seed(42)

# CuDNN 관련 설정 (GPU 연산의 재현성을 위함)
# 결정론적 알고리즘을 사용하도록 설정하여 실행 시마다 동일한 결과를 보장
torch.backends.cudnn.deterministic = True
# 내장된 벤치마크 기능을 비활성화하여 재현성 유지
torch.backends.cudnn.benchmark = False

# Linear Probing

## 준비과정

In [2]:
# --- 필수 라이브러리 임포트 ---
import torch
import torch.nn as nn                       # 신경망 모델을 구성하기 위한 모듈 (레이어, 손실 함수 등)
import torch.optim as optim                 # 모델을 최적화하기 위한 알고리즘 (SGD, Adam 등)
import torchvision                          # 컴퓨터 비전 관련 유명 데이터셋, 모델, 변환 기능을 제공
import torchvision.transforms as transforms # 이미지 데이터를 전처리(변환)하기 위한 기능
from tqdm import tqdm

# --- 장치 설정 ---
# torch.cuda.is_available() 함수는 GPU 사용이 가능한지 확인
# 가능하면 'cuda' (GPU)를, 불가능하면 'cpu'를 device 변수에 할당
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 중인 장치:", device)



# CIFAR-10 데이터셋 로드 (train 50,000장, test 10,000장)
# --- CIFAR-10 데이터셋 로드 ---
# root: 데이터가 저장될 경로
# train=True: 훈련용 데이터셋을 불러옴
# download=True: 해당 경로에 데이터가 없으면 다운로드
# transform: 데이터 변환기 적용 -> 데이터를 먼저 확인하기 위해 False 으로 설정
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=False)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=False)

print(trainset.data.shape, testset.data.shape)


# CIFAR_10 데이터셋의 평균과 표준편차 계산
data_tensor = torch.from_numpy(trainset.data)

# 평균 계산 (배치, 높이, 너비 차원에 대해)
mean = torch.mean(data_tensor.float() / 255.0, dim=(0, 1, 2))

# 표준편차 계산 (배치, 높이, 너비 차원에 대해)
std = torch.std(data_tensor.float() / 255.0, dim=(0, 1, 2))

print("CIFAR-10 학습 데이터의 픽셀별 평균:", mean)
print("CIFAR-10 학습 데이터의 픽셀별 표준편차:", std)

사용 중인 장치: cuda


100%|██████████| 170M/170M [00:03<00:00, 49.1MB/s]


(50000, 32, 32, 3) (10000, 32, 32, 3)
CIFAR-10 학습 데이터의 픽셀별 평균: tensor([0.4914, 0.4822, 0.4465])
CIFAR-10 학습 데이터의 픽셀별 표준편차: tensor([0.2470, 0.2435, 0.2616])


### 정규화

In [4]:
# --- 데이터 변환기(Transformer) 정의 ---
# 이미지에 적용할 전처리 단계를 Compose를 사용해 묶어줍니다.

# 학습 데이터: 224로 리사이즈 후 텐서화 및 정규화
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),      # ResNet18 입력 크기인 224x224로 크기 변경
    transforms.ToTensor(),              # PyTorch 텐서로 변환

    # 픽셀 값을 특정 평균(mean)과 표준편차(std)로 정규화
    transforms.Normalize(mean,  # CIFAR-10 평균
                         std)   # CIFAR-10 표준편차
])

# 문제 1: 데이터 전처리 과정을 위한 변환기(Transformer)를 정의하세요.
# 테스트 데이터: 리사이즈 후 텐서화 및 정규화 (학습과 동일하게)
test_transform = transforms.Compose([
    # [START CODE]
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    # [END CODE]
])

# 위에서 정의한 변환기를 사용하여 CIFAR-10 데이터셋 로드 (train 50,000장, test 10,000장)
# root: 데이터가 저장될 경로
# train=True: 훈련용 데이터셋을 불러옴
# download=True: 해당 경로에 데이터가 없으면 다운로드
# transform: 위에서 정의한 데이터 변환기 적용
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

# --- DataLoader 생성 ---
# DataLoader는 데이터를 미니배치(mini-batch) 단위로 묶어주는 역할을 함
# batch_size: 한 번에 모델에 입력할 데이터(이미지)의 개수
# shuffle=True: 훈련 시 데이터를 무작위로 섞어 모델이 데이터 순서에 과적합되는 것을 방지
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True)
testloader  = torch.utils.data.DataLoader(testset, batch_size=256, shuffle=False)

print("훈련 배치 개수:", len(trainloader), "테스트 배치 개수:", len(testloader))


훈련 배치 개수: 196 테스트 배치 개수: 40


### ResNet 가져오기

### 선형 계층만 학습되도록 설정

In [9]:
model = torchvision.models.resnet18(pretrained=True)

# --- 선형 프로빙(Linear Probing)을 위한 모델 수정 ---

# 문제 2: 선형 프로빙을 위해 ResNet-18 모델의 마지막 층을 수정하세요.
# 1. 마지막 분류층(Fully Connected Layer) 교체
# 기존 ResNet-18의 마지막 층(model.fc)은 ImageNet의 1000개 클래스를 분류
# 이를 CIFAR-10의 10개 클래스를 분류하도록 새로운 선형 레이어로 교체
# model.fc.in_features는 기존 fc 레이어의 입력 뉴런 수를 그대로 사용
# [START CODE]
model.fc = nn.Linear(model.fc.in_features, 10)
# [END CODE]

# 문제 3: 선형 프로빙을 위해 ResNet-18 모델의 마지막 층을 업데이트가 되지 않도록 가중치를 동결하세요.
# 2. 마지막 층을 제외한 모든 파라미터(가중치)를 동결(freeze)
# == 최종 fc 레이어를 제외한 모든 파라미터를 동결
# `param.requires_grad = False`로 설정하면 해당 파라미터는 학습 중에 업데이트되지 않음
# [START CODE]
for name, param in model.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False

# [END CODE]

# 장치를 GPU로 이동
model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### 학습 시작

In [10]:
# 문제 4: 손실 함수를 정의하세요
# 손실 함수로 CrossEntropyLoss를 정의
# [START CODE]
criterion = nn.CrossEntropyLoss()
# [END CODE]

# 문제 5: 옵티마이저를 정의하세요
# 옵티마이저로 SGD(Stochastic Gradient Descent)를 정의, 학습률은 0.001 으로 설정
# 동결된 파라미터 제외, 마지막 레이어만 최적화
# 즉, 마지막 레이어의 파라미터만 옵티마이저에 전달하여 마지막 레이어만 최적화되도록 설정
# [START CODE]
optimizer = optim.SGD(model.parameters(), lr=0.001)
# [END CODE]

# 에포크(epoch) 수 설정. 에포크는 전체 훈련 데이터를 한 번 모두 사용하는 것을 의미
num_epochs = 5

# 문제 6: 모델 학습을 위한 반복문을 작성하세요
for epoch in range(num_epochs):
    model.train()       # 모델을 학습 모드로 설정
    running_loss = 0.0  # 에포크 동안의 총 손실을 기록할 변수

    # trainloader에서 미니배치 단위로 데이터를 가져와 반복
    for inputs, labels in tqdm(trainloader):

        # 입력 데이터와 정답 레이블을 지정된 장치(GPU)로 이동
        inputs, labels = inputs.to(device), labels.to(device)

        # --- 핵심 학습 단계 ---
        # [START CODE]

        # 1. 옵티마이저의 그래디언트(gradient)를 0으로 초기화
        # 이전 배치의 그래디언트가 다음 배치에 영향을 주지 않도록 함
        optimizer.zero_grad()

        # 2. 모델에 입력을 넣어 순전파(forward pass) 진행 및 출력(outputs) 계산
        outputs = model(inputs)

        # 3. 모델의 출력과 실제 정답을 비교하여 손실(loss) 계산
        loss = criterion(outputs, labels)

        # 4. 역전파(backward pass)를 통해 각 파라미터에 대한 그래디언트 계산
        loss.backward()

        # 5. 옵티마이저를 사용해 모델의 파라미터(가중치)를 업데이트
        # requires_grad=True로 설정된 파라미터만 업데이트됨 (여기서는 fc 층만)
        optimizer.step()

        # [END CODE]

        # 현재 배치의 손실을 running_loss에 더함
        running_loss += loss.item()

    # 전체 에포크가 끝난 후 평균 훈련 손실을 계산하고 출력
    avg_loss = running_loss / len(trainloader)
    print(f"[Epoch {epoch+1}/{num_epochs}] 평균 훈련 손실: {avg_loss:.4f}")


100%|██████████| 196/196 [02:26<00:00,  1.34it/s]


[Epoch 1/5] 평균 훈련 손실: 2.1578


100%|██████████| 196/196 [02:31<00:00,  1.29it/s]


[Epoch 2/5] 평균 훈련 손실: 1.8242


100%|██████████| 196/196 [02:26<00:00,  1.34it/s]


[Epoch 3/5] 평균 훈련 손실: 1.5936


100%|██████████| 196/196 [02:27<00:00,  1.33it/s]


[Epoch 4/5] 평균 훈련 손실: 1.4279


100%|██████████| 196/196 [02:21<00:00,  1.38it/s]

[Epoch 5/5] 평균 훈련 손실: 1.3072


### 학습 결과 확인

In [11]:
# 모델을 평가 모드로 설정
# 이 모드에서는 드롭아웃(Dropout)이나 배치 정규화(Batch Normalization) 등이 비활성화되어 일관된 예측 결과를 얻을 수 있음
model.eval()

correct = 0   # 맞춘 예측 개수
total = 0     # 전체 데이터 개수

# 그래디언트 계산을 비활성화하는 컨텍스트
with torch.no_grad():

    # testloader에서 미니배치 단위로 데이터를 가져와 반복
    for inputs, labels in tqdm(testloader):

        # 입력 데이터와 정답 레이블을 장치(GPU)로 이동
        inputs, labels = inputs.to(device), labels.to(device)

        # 모델에 입력을 넣어 출력 계산
        outputs = model(inputs)

        # 문제 7: 예측 및 정확도를 계산하세요
        # [START CODE]
        # --- 예측 및 정확도 계산 ---
        # outputs.data는 모델의 최종 출력(logits)
        # torch.max(..., 1)은 각 샘플에 대해 가장 높은 값과 그 인덱스를 반환
        # `_`는 값(무시), `predicted`는 인덱스(예측된 클래스)
        _, predicted = torch.max(outputs.data, 1)
        # [END CODE]

        total += labels.size(0) # 현재 배치의 데이터 개수를 total에 더함

        # 예측이 정답과 일치하는 개수를 세어 correct에 더함
        correct += (predicted == labels).sum().item()

# 전체 정확도 계산 및 출력
accuracy = 100 * correct / total
print(f"테스트 데이터 정확도: {accuracy:.2f}%")

100%|██████████| 40/40 [00:27<00:00,  1.47it/s]

테스트 데이터 정확도: 66.80%


## 데이터 증강(Data Augmentation) & Fine Tuning

In [12]:
# 문제 8: 데이터 증강을 적용한 변환기를 정의하세요
# --- 데이터 증강이 포함된 변환기 정의 ---
train_transform_aug = transforms.Compose([
    # [START CODE]
    # 32x32 이미지 주변에 4픽셀의 패딩(padding)을 추가한 뒤, 무작위로 32x32 영역을 잘라냄
    transforms.RandomCrop(32, padding=4),
    # 50% 확률로 이미지를 좌우로 뒤집음
    transforms.RandomHorizontalFlip(p=0.5),
    # [END CODE]
    transforms.Resize((224, 224)), # 이미지 크기 조절
    transforms.ToTensor(), # 텐서로 변환
    transforms.Normalize(mean=mean, std=std) # 정규화
])

# 데이터 증강이 적용된 새로운 훈련 데이터셋 및 DataLoader 생성
trainset_aug = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform_aug)
trainloader_aug = torch.utils.data.DataLoader(trainset_aug, batch_size=256, shuffle=True)

In [13]:
# 문제 9: 미세 조정을 위해 모델 파라미터 동결을 해제하세요
# [START CODE]
# --- 모델의 모든 파라미터 동결 해제 ---
# requires_grad를 True로 설정하여 모든 파라미터가 학습 중에 업데이트되도록 함
for param in model.parameters():
    param.requires_grad = True

# [END CODE]

# --- 새로운 옵티마이저와 스케줄러 정의 ---
# 이제 model.parameters()를 전달하여 모델의 모든 파라미터를 최적화 대상으로 함
# 학습률(lr)은 기존보다 약간 낮게 설정하여 섬세하게 조정
optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

# StepLR 스케줄러: 특정 단계(step_size)마다 학습률에 감마(gamma)를 곱해 감소시킴
# step_size=5: 5 에포크마다
# gamma=0.1: 학습률을 0.1배로 줄임
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

### 학습 시작

In [14]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train() # 훈련 모드
    running_loss = 0.0

    # 증강이 적용된 데이터로 훈련
    for inputs, labels in tqdm(trainloader_aug):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # 에포크 결과 출력
    avg_loss = running_loss / len(trainloader_aug)
    print(f"[Fine-tune Epoch {epoch+1}/{num_epochs}] 평균 훈련 손실: {avg_loss:.4f}, 현재 학습률: {optimizer.param_groups[0]['lr']:.6f}")

    # 문제 10: 학습률 스케줄러를 통해 학습률을 조정하세요
    # [START CODE]
    # --- 학습률 스케줄러 업데이트 ---
    # 정의된 규칙에 따라 학습률을 조정
    scheduler.step()

    # [END CODE]

100%|██████████| 196/196 [03:54<00:00,  1.20s/it]


[Fine-tune Epoch 1/5] 평균 훈련 손실: 0.7636, 현재 학습률: 0.000500


100%|██████████| 196/196 [03:54<00:00,  1.19s/it]


[Fine-tune Epoch 2/5] 평균 훈련 손실: 0.4149, 현재 학습률: 0.000500


100%|██████████| 196/196 [03:55<00:00,  1.20s/it]


[Fine-tune Epoch 3/5] 평균 훈련 손실: 0.3223, 현재 학습률: 0.000500


100%|██████████| 196/196 [03:55<00:00,  1.20s/it]


[Fine-tune Epoch 4/5] 평균 훈련 손실: 0.2755, 현재 학습률: 0.000500


100%|██████████| 196/196 [03:54<00:00,  1.20s/it]

[Fine-tune Epoch 5/5] 평균 훈련 손실: 0.2455, 현재 학습률: 0.000500


In [17]:
# --- 미세 조정 후 모델 평가 ---
model.eval() # 평가 모드
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in tqdm(testloader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

fine_tune_acc = 100 * correct / total
print(f"미세 조정 후 테스트 정확도: {fine_tune_acc:.2f}%")

100%|██████████| 40/40 [00:28<00:00,  1.42it/s]

미세 조정 후 테스트 정확도: 92.27%


In [18]:
torch.save(model.state_dict(), 'model.pth')